In [1]:
import wfdb
import os
import pickle
import numpy as np

In [2]:
def load_training_data(sample_path):
    record = wfdb.rdrecord(sample_path)
    ann_ref = wfdb.rdann(sample_path, 'atr')
    fields = record.__dict__
    sig = fields['p_signal']
    length = fields['sig_len']
    fs = fields['fs']
    comments = fields['comments'][0]
    if comments == 'paroxysmal atrial fibrillation':
        class_true = 2
    elif comments == 'persistent atrial fibrillation':
        class_true = 1
    else:
        class_true = 0
        
    record_name = fields['record_name']
    
    # y
    
    beat_loc = np.array(ann_ref.sample)
    ann_note = np.array(ann_ref.aux_note)
    af_start_scripts = np.where((ann_note=='(AFIB') | \
                                (ann_note=='(AFL'))[0]
    af_end_scripts = np.where(ann_note=='(N')[0]

    return record_name, sig, length, fs, beat_loc, af_start_scripts, af_end_scripts, class_true

In [3]:
DATA_PATH = ['../../ECG Recordings/Training_set_I', '../../ECG Recordings/Training_set_II']
training_data_I = os.listdir(DATA_PATH[0])
training_data_II = os.listdir(DATA_PATH[1])

samples = [[], []]
for data in training_data_I:
    samples[0].append(data[:-4])
for data in training_data_II:
    samples[1].append(data[:-4])
samples = [list(set(samples[0])), list(set(samples[1]))]

In [4]:
training_data = []
for s in samples[0]:
    record_name, sig, length, fs, beat_loc, \
        af_start_scripts, af_end_scripts, class_true = \
            load_training_data(DATA_PATH[0] + '/' + s)
    data = {"record_name": record_name, "sig": sig, "sig_len": length, "fs": fs, "beat_loc": beat_loc, "af_start_scripts": af_start_scripts, "af_end_scripts": af_end_scripts, "class_true": class_true}
    training_data.append(data)
    
for s in samples[1]:
    record_name, sig, length, fs, beat_loc, \
        af_start_scripts, af_end_scripts, class_true = \
            load_training_data(DATA_PATH[1] + '/' + s)
    data = {"record_name": record_name, "sig": sig, "sig_len": length, "fs": fs, "beat_loc": beat_loc, "af_start_scripts": af_start_scripts, "af_end_scripts": af_end_scripts, "class_true": class_true}
    training_data.append(data)

In [5]:
with open('../data/data.pkl', 'wb') as file:
    pickle.dump(training_data, file)